# Google Cloud Functions for Deep Learning with Earth Engine
**Setup software libraries**

In [1]:
# Import and initialize the Earth Engine library.
import ee
ee.Initialize()
ee.__version__

'0.1.202'

In [2]:
# Folium setup.
import folium
print(folium.__version__)

0.8.3


In [3]:
import ee_collection_specifics
from pprint import pprint
import requests
import json
import env

## Data pre-processing
### Python package with the cloud function code

It's necessary to create a Python package to hold the cloud function code.  Here we're going to get started with that by creating a folder for the package.

In [65]:
PACKAGE_PATH = 'Google_Cloud_Functions/ee_pre_processing'

!rm -r {PACKAGE_PATH}
!mkdir {PACKAGE_PATH}
!ls -l {PACKAGE_PATH}

**Files**

Fist we copy the `privatekey.json` file with the [service account keys](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) and the `ee_collection_specifics.py` and `env.py` files into de folder.

In [66]:
!cp ee_collection_specifics.py {PACKAGE_PATH}/ee_collection_specifics.py
!cp {env.privatekey_path} {PACKAGE_PATH}/privatekey.json
!cp env.py {PACKAGE_PATH}/env.py

**`main.py` file**

We'll use the `%%writefile` command to write the contents of the main function code to a file called `main.py`.

In [67]:
%%writefile {PACKAGE_PATH}/main.py

import ee
import json
import numpy as np
import ee_collection_specifics
import env

account = env.service_account
credentials = ee.ServiceAccountCredentials(account, 'privatekey.json')
ee.Initialize(credentials)

def min_max_values(image, collection, scale):
    
    normThreshold = ee_collection_specifics.ee_bands_normThreshold(collection)
    
    num = 2
    lon = np.linspace(-180, 180, num)
    lat = np.linspace(-90, 90, num)
    
    features = []
    for i in range(len(lon)-1):
        for j in range(len(lat)-1):
            features.append(ee.Feature(ee.Geometry.Rectangle(lon[i], lat[j], lon[i+1], lat[j+1])))
    
    regReducer = {
        'geometry': ee.FeatureCollection(features),
        'reducer': ee.Reducer.minMax(),
        'maxPixels': 1e10,
        'bestEffort': True,
        'scale':scale
        
    }
    
    values = image.reduceRegion(**regReducer).getInfo()
    print(values)
    
    # Avoid outliers by taking into account only the normThreshold% of the data points.
    regReducer = {
        'geometry': ee.FeatureCollection(features),
        'reducer': ee.Reducer.histogram(),
        'maxPixels': 1e10,
        'bestEffort': True,
        'scale':scale
        
    }
    
    hist = image.reduceRegion(**regReducer).getInfo()

    for band in list(normThreshold.keys()):
        if normThreshold[band] != 100:
            count = np.array(hist.get(band).get('histogram'))
            x = np.array(hist.get(band).get('bucketMeans'))
        
            cumulative_per = np.cumsum(count/count.sum()*100)
        
            values[band+'_max'] = x[np.where(cumulative_per < normThreshold[band])][-1]
        
    return values

def normalize_ee_images(image, collection, values):
    
    Bands = ee_collection_specifics.ee_bands(collection)
       
    # Normalize [0, 1] ee images
    for i, band in enumerate(Bands):
        if i == 0:
            image_new = image.select(band).clamp(values[band+'_min'], values[band+'_max'])\
                                .subtract(values[band+'_min'])\
                                .divide(values[band+'_max']-values[band+'_min'])
        else:
            image_new = image_new.addBands(image.select(band).clamp(values[band+'_min'], values[band+'_max'])\
                                    .subtract(values[band+'_min'])\
                                    .divide(values[band+'_max']-values[band+'_min']))
            
    return image_new
    
def ee_pre_processing(request):
    request = request.get_json()

    # Variables
    collection = request.get('collection')
    startDate = ee.Date(request.get('start'))
    stopDate  = ee.Date(request.get('end'))
    scale  = request.get('scale')
    # Bands
    bands = ee_collection_specifics.ee_bands(collection)

    # Get composite
    image = ee_collection_specifics.Composite(collection)(startDate, stopDate)
    image = image.select(bands)

    # Normalize images
    if ee_collection_specifics.normalize(collection):
        # Get min man values for each band
        values = min_max_values(image, collection, scale)

        # Normalize images
        image = normalize_ee_images(image, collection, values)
        
    return json.dumps({'bands_min_max': values, 'composite': image.serialize()})

Writing Google_Cloud_Functions/ee_pre_processing/main.py


We include the `requirements.txt` file:

In [68]:
%%writefile {PACKAGE_PATH}/requirements.txt
earthengine-api==0.1.202
numpy==1.17.2

Writing Google_Cloud_Functions/ee_pre_processing/requirements.txt


Finally cd to that directory and deploy the cloud Function with the following command:

`gcloud functions deploy ee_pre_processing --runtime python37 --trigger-http --timeout=400`

In [69]:
%cd {PACKAGE_PATH}
!gcloud functions deploy ee_pre_processing --runtime python37 --trigger-http --timeout=400

/Users/ikersanchez/Vizzuality/GitHub/Skydipper/CNN-tests/notebooks/Google_Cloud_Functions/ee_pre_processing
Deploying function (may take a while - up to 2 minutes)...done.                
availableMemoryMb: 256
entryPoint: ee_pre_processing
httpsTrigger:
  url: https://us-central1-skydipper-196010.cloudfunctions.net/ee_pre_processing
labels:
  deployment-tool: cli-gcloud
name: projects/skydipper-196010/locations/us-central1/functions/ee_pre_processing
runtime: python37
serviceAccountEmail: skydipper-196010@appspot.gserviceaccount.com
sourceUploadUrl: https://storage.googleapis.com/gcf-upload-us-central1-6055a6ac-90d7-4296-b9d7-f9a2b2e4cff2/ddf37c5b-93f0-4316-bac9-52a65ee0c025.zip?GoogleAccessId=service-230510979472@gcf-admin-robot.iam.gserviceaccount.com&Expires=1576684323&Signature=iUpBGbR6HQZpDoVeveC7Npmd%2FFd2AVm8LuANXDYeOl4sstdCKpafxjaV60PzlhOQ7vfsV0Rv7RvT3aOY3ImhuV2iSuCBjiFN0AOT1UEQMnoXrUtxboAdYlnomJScS6v3F6VUrXLJ%2F%2Fhymrt85O7%2B3JYgUBEDoXzRnHlJFfEjL2BpDVxvrCQ6R2vWHAuBURAQb9cHkj

cd back to the current working directory

In [56]:
%cd .. 
%cd .. 

/Users/ikersanchez/Vizzuality/GitHub/Skydipper/CNN-tests/notebooks/Google_Cloud_Functions
/Users/ikersanchez/Vizzuality/GitHub/Skydipper/CNN-tests/notebooks
AI_Platform
Cloud_Optimized_Geotiffs.ipynb
DL_with_EE_CloudStorage_AIPlatform.ipynb
Download_Earth_Engine.ipynb
EE_as_a_tile_server.ipynb
Earth_Engine_and_Keras.ipynb
GCF_for_DL_in_EE.ipynb
Google_Cloud_Functions
Keras_in_AI_Platform_TFRecords.ipynb
Keras_in_ML_Engine.ipynb
Keras_with_TFData.ipynb
Keras_with_TFRecord.ipynb
SegNet_Keras.ipynb
Segmentation_TFRecords_ML_Engine.ipynb
Sentinel2_classifier.ipynb
Sentinel2_cropland_mapping.ipynb
Tile_Prediction.ipynb
__pycache__
config.yaml
ee_collection_specifics.py
env.py
preprocess


### Call

In [61]:
payload =   {
    "collection": 'Sentinel2_TOA',
    "start": '2016-01-01',
    "end": '2016-12-31',
    "scale": 30
}

In [62]:
url = f'https://us-central1-skydipper-196010.cloudfunctions.net/ee_pre_processing'

headers = {'Content-Type': 'application/json'}

output_pre_processing = requests.post(url, data=json.dumps(payload), headers=headers)
output_pre_processing.json()

{'composite': '{"type": "CompoundValue", "scope": [["0", {"type": "Invocation", "arguments": {"id": "COPERNICUS/S2"}, "functionName": "ImageCollection.load"}], ["1", {"type": "Invocation", "arguments": {"value": "2016-01-01"}, "functionName": "Date"}], ["2", {"type": "Invocation", "arguments": {"value": "2016-12-31"}, "functionName": "Date"}], ["3", {"type": "Invocation", "arguments": {"start": {"type": "ValueRef", "value": "1"}, "end": {"type": "ValueRef", "value": "2"}}, "functionName": "DateRange"}], ["4", {"type": "Invocation", "arguments": {"rightField": "system:time_start", "leftValue": {"type": "ValueRef", "value": "3"}}, "functionName": "Filter.dateRangeContains"}], ["5", {"type": "Invocation", "arguments": {"collection": {"type": "ValueRef", "value": "0"}, "filter": {"type": "ValueRef", "value": "4"}}, "functionName": "Collection.filter"}], ["6", {"type": "Invocation", "arguments": {"leftField": "CLOUDY_PIXEL_PERCENTAGE", "rightValue": 20}, "functionName": "Filter.lessThan"}],

**Deserialize**

In [63]:
image = ee.deserializer.fromJSON(output_pre_processing.json()['composite'])

**Display**

In [64]:
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
# Use folium to visualize the imagery.
collection = payload.get("collection")
map = folium.Map(location=[38.1623, -121.6911])
for params in ee_collection_specifics.vizz_params(collection):
    mapid = image.getMapId(params)
    folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name=str(params['bands']),
  ).add_to(map)
    
map.add_child(folium.LayerControl())
map

***
## Create TFRecords for training
### Python package with the cloud function code

It's necessary to create a Python package to hold the cloud function code.  Here we're going to get started with that by creating a folder for the package.

In [12]:
PACKAGE_PATH = 'Google_Cloud_Functions/ee_tfrecords_training'

!rm -r {PACKAGE_PATH}
!mkdir {PACKAGE_PATH}
!ls -l {PACKAGE_PATH}

**Files**

Fist we copy the `privatekey.json` file with the [service account keys](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) and the `ee_collection_specifics.py` and `env.py` files into the folder.

In [13]:
!cp ee_collection_specifics.py {PACKAGE_PATH}/ee_collection_specifics.py
!cp {env.privatekey_path} {PACKAGE_PATH}/privatekey.json
!cp env.py {PACKAGE_PATH}/env.py

**`main.py` file**

We'll use the `%%writefile` command to write the contents of the main function code to a file called `main.py`.

In [14]:
%%writefile {PACKAGE_PATH}/main.py

import ee
import json
import numpy as np
import requests
import ee_collection_specifics
import env

account = env.service_account
credentials = ee.ServiceAccountCredentials(account, 'privatekey.json')
ee.Initialize(credentials)

def image_into_array(url, collections, bands, kernelSize, startDate, stopDate, scale):

    headers = {'Content-Type': 'application/json'}
    
    for i, collection in enumerate(collections):
        payload =   {
            "collection": collection,
            "start": startDate,
            "end": stopDate,
            "scale": scale
        }
        
        output = requests.post(url, data=json.dumps(payload), headers=headers)
        
        if i == 0:
            image = ee.deserializer.fromJSON(output.json()['composite']).select(bands[i])
        else:
            featureStack = ee.Image.cat([image,\
                                         ee.deserializer.fromJSON(output.json()['composite']).select(bands[i])\
                                        ]).float()
            
    list = ee.List.repeat(1, kernelSize)
    lists = ee.List.repeat(list, kernelSize)
    kernel = ee.Kernel.fixed(kernelSize, kernelSize, lists)
    
    arrays = featureStack.neighborhoodToArray(kernel)
    
    return arrays

def GeoJSONs_to_FeatureCollections(multipolygon):
    # Make a list of Features
    features = []
    for i in range(len(multipolygon.get('features')[0].get('geometry').get('coordinates'))):
        features.append(
            ee.Feature(
                ee.Geometry.Polygon(
                    multipolygon.get('features')[0].get('geometry').get('coordinates')[i]
                )
            )
        )
        
    # Create a FeatureCollection from the list and print it.
    return ee.FeatureCollection(features)

def export_TFRecords(arrays, scale, nShards, sampleSize, features, polysLists, baseNames, bucket, folder, selectors):
    # Export all the training/evaluation data (in many pieces), with one task per geometry.
    filePaths = []
    for i, feature in enumerate(features):
        for g in range(feature.size().getInfo()):
            geomSample = ee.FeatureCollection([])
            for j in range(nShards):
                sample = arrays.sample(
                    region = ee.Feature(polysLists[i].get(g)).geometry(), 
                    scale = scale, 
                    numPixels = sampleSize / nShards, # Size of the shard.
                    seed = j,
                    tileScale = 8
                )
                geomSample = geomSample.merge(sample)
                
            desc = baseNames[i] + '_g' + str(g)
            
            filePaths.append(bucket+ '/' + folder + '/' + desc)
            
            task = ee.batch.Export.table.toCloudStorage(
                collection = geomSample,
                description = desc, 
                bucket = bucket, 
                fileNamePrefix = folder + '/' + desc,
                fileFormat = 'TFRecord',
                selectors = selectors
            )
            task.start()
            
    return filePaths   

def ee_tfrecords_training(request):
    request = request.get_json()

    # Variables
    inCollection = request.get('in_collection')
    outCollection = request.get('out_collection')
    inBands = request.get('in_bands')
    outBands = request.get('out_bands')
    startDate = request.get('start')
    stopDate = request.get('end')
    scale = request.get('scale')
    sampleSize = request.get('sample_size')
    datasetName = request.get('dataset_name')
    trainPolys = request.get('train_polys')
    evalPolys = request.get('eval_polys')

    # An array of images
    url = f'https://us-central1-skydipper-196010.cloudfunctions.net/ee_pre_processing'
    collections = [inCollection, outCollection]
    bands = [inBands, outBands]
    kernelSize = 256

    arrays = image_into_array(url, collections, bands, kernelSize, startDate, stopDate, scale)
    
    # Convert the GeoJSONs to feature collections
    trainFeatures = GeoJSONs_to_FeatureCollections(trainPolys)
    evalFeatures = GeoJSONs_to_FeatureCollections(evalPolys)

    # Convert the feature collections to lists for iteration.
    trainPolysList = trainFeatures.toList(trainFeatures.size())
    evalPolysList = evalFeatures.toList(evalFeatures.size())

    # These numbers determined experimentally.
    nShards  = int(sampleSize/20)#100 # Number of shards in each polygon.

    features = [trainFeatures, evalFeatures]
    polysLists = [trainPolysList, evalPolysList]
    baseNames = ['training_patches', 'eval_patches']
    bucket = 'skydipper_materials'
    folder = 'cnn-models/'+datasetName+'/data'
    selectors = inBands + outBands

    # Export all the training/evaluation data (in many pieces), with one task per geometry.
    filePaths   = export_TFRecords(arrays, scale, nShards, sampleSize, features, polysLists, baseNames, bucket, folder, selectors)    

        
    return json.dumps({
      "file_paths": filePaths, 
      "training_polygons": trainFeatures.serialize(), 
      "evaluation_polygons": evalFeatures.serialize()
      })

Writing Google_Cloud_Functions/ee_tfrecords_training/main.py


We include the `requirements.txt` file:

In [15]:
%%writefile {PACKAGE_PATH}/requirements.txt
earthengine-api==0.1.202
numpy==1.17.2
requests==2.21.0

Writing Google_Cloud_Functions/ee_tfrecords_training/requirements.txt


Finally cd to that directory and deploy the cloud Function with the following command:

`gcloud functions deploy ee_tfrecords_training --runtime python37 --trigger-http --timeout=400`

#### Call

In [ ]:
payload = {
    "in_collection": 'Landsat8_SR',
    "out_collection": 'CroplandDataLayers',
    "in_bands": ['B1','B2','B3','B4','B5','B6','B7'],
    "out_bands": ['cropland', 'land', 'water', 'urban'],
    "start": '2016-01-01',
    "end": '2016-12-31',
    "scale": 30, 
    "sample_size": 10,
    "dataset_name": 'Landsat8_Cropland3',
    "train_polys": {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "MultiPolygon",
            "coordinates":  [
                [[[  -122.882080078125,  40.50126945841645],[  -122.1240234375,  40.50126945841645],[  -122.1240234375,  41.008920735004885],[  -122.882080078125,  41.008920735004885],[  -122.882080078125,  40.50126945841645]]],
                [[[  -122.2283935546875,  39.00637903337455],[  -121.607666015625,  39.00637903337455],[  -121.607666015625,  39.46588451142044],[  -122.2283935546875,  39.46588451142044],[  -122.2283935546875,  39.00637903337455]]],
                [[[  -120.355224609375,  38.77978137804918],[  -119.608154296875,  38.77978137804918],[  -119.608154296875,  39.342794408952365],[  -120.355224609375,  39.342794408952365],[  -120.355224609375,  38.77978137804918]]],
                [[[  -121.90979003906249,  37.70555348721583],[  -120.9814453125,  37.70555348721583],[  -120.9814453125,  38.39764411353178],[  -121.90979003906249,  38.39764411353178],[  -121.90979003906249,  37.70555348721583]]],
                [[[  -120.03662109374999,  37.45741810262938],[  -119.1851806640625,  37.45741810262938],[  -119.1851806640625,  38.08268954483802],[  -120.03662109374999,  38.08268954483802],[  -120.03662109374999,  37.45741810262938]]],
                [[[  -120.03662109374999,  37.45741810262938],[  -119.1851806640625,  37.45741810262938],[  -119.1851806640625,  38.08268954483802],[  -120.03662109374999,  38.08268954483802],[  -120.03662109374999,  37.45741810262938]]],
                [[[  -120.03662109374999,  37.45741810262938],[  -119.1851806640625,  37.45741810262938],[  -119.1851806640625,  38.08268954483802],[  -120.03662109374999,  38.08268954483802],[  -120.03662109374999,  37.45741810262938]]],
                [[[  -112.554931640625,  33.0178760185549],[  -111.588134765625,  33.0178760185549],[  -111.588134765625,  33.78827853625996],[  -112.554931640625,  33.78827853625996],[  -112.554931640625,  33.0178760185549]]],
                [[[  -112.87353515625,  40.51379915504413],[  -111.829833984375,  40.51379915504413],[  -111.829833984375,  41.28606238749825],[  -112.87353515625,  41.28606238749825],[  -112.87353515625,  40.51379915504413]]],
                [[[  -108.19335937499999,  39.095962936305476],[  -107.1826171875,  39.095962936305476],[  -107.1826171875,  39.85915479295669],[  -108.19335937499999,  39.85915479295669],[  -108.19335937499999,  39.095962936305476]]],
                [[[  -124.25537109375,  30.86451022625836],[  -124.25537109375,  30.86451022625836],[  -124.25537109375,  30.86451022625836],[  -124.25537109375,  30.86451022625836]]],
                [[[  -106.875,  37.142803443716836],[  -105.49072265625,  37.142803443716836],[  -105.49072265625,  38.18638677411551],[  -106.875,  38.18638677411551],[  -106.875,  37.142803443716836]]],
                [[[  -117.31201171875001,  43.27720532212024],[  -116.01562499999999,  43.27720532212024],[  -116.01562499999999,  44.134913443750726],[  -117.31201171875001,  44.134913443750726],[  -117.31201171875001,  43.27720532212024]]],
                [[[  -115.7080078125,  44.69989765840318],[  -114.7412109375,  44.69989765840318],[  -114.7412109375,  45.36758436884978],[  -115.7080078125,  45.36758436884978],[  -115.7080078125,  44.69989765840318]]],
                [[[  -120.65185546875,  47.517200697839414],[  -119.33349609375,  47.517200697839414],[  -119.33349609375,  48.32703913063476],[  -120.65185546875,  48.32703913063476],[  -120.65185546875,  47.517200697839414]]],
                [[[  -119.83886718750001,  45.69083283645816],[  -118.38867187500001,  45.69083283645816],[  -118.38867187500001,  46.694667307773116],[  -119.83886718750001,  46.694667307773116],[  -119.83886718750001,  45.69083283645816]]],
                [[[  -107.09472656249999,  47.45780853075031],[  -105.84228515625,  47.45780853075031],[  -105.84228515625,  48.31242790407178],[  -107.09472656249999,  48.31242790407178],[  -107.09472656249999,  47.45780853075031]]],
                [[[  -101.57958984375,  46.93526088057719],[  -100.107421875,  46.93526088057719],[  -100.107421875,  47.945786463687185],[  -101.57958984375,  47.945786463687185],[  -101.57958984375,  46.93526088057719]]],
                [[[  -101.162109375,  44.32384807250689],[  -99.7119140625,  44.32384807250689],[  -99.7119140625,  45.22848059584359],[  -101.162109375,  45.22848059584359],[  -101.162109375,  44.32384807250689]]],
                [[[  -100.5908203125,  41.261291493919884],[  -99.25048828124999,  41.261291493919884],[  -99.25048828124999,  42.114523952464246],[  -100.5908203125,  42.114523952464246],[  -100.5908203125,  41.261291493919884]]],
                [[[  -97.9541015625,  37.142803443716836],[  -96.65771484375,  37.142803443716836],[  -96.65771484375,  38.13455657705411],[  -97.9541015625,  38.13455657705411],[  -97.9541015625,  37.142803443716836]]],
                [[[  -112.78564453124999,  32.91648534731439],[  -111.357421875,  32.91648534731439],[  -111.357421875,  33.925129700072],[  -112.78564453124999,  33.925129700072],[  -112.78564453124999,  32.91648534731439]]],
                [[[  -106.435546875,  35.15584570226544],[  -105.22705078125,  35.15584570226544],[  -105.22705078125,  36.13787471840729],[  -106.435546875,  36.13787471840729],[  -106.435546875,  35.15584570226544]]],
                [[[  -97.3828125,  32.45415593941475],[  -96.2841796875,  32.45415593941475],[  -96.2841796875,  33.22949814144951],[  -97.3828125,  33.22949814144951],[  -97.3828125,  32.45415593941475]]],
                [[[  -97.97607421875,  35.04798673426734],[  -97.00927734375,  35.04798673426734],[  -97.00927734375,  35.764343479667176],[  -97.97607421875,  35.764343479667176],[  -97.97607421875,  35.04798673426734]]],
                [[[  -97.97607421875,  35.04798673426734],[  -97.00927734375,  35.04798673426734],[  -97.00927734375,  35.764343479667176],[  -97.97607421875,  35.764343479667176],[  -97.97607421875,  35.04798673426734]]],
                [[[  -95.4052734375,  47.62097541515849],[  -94.24072265625,  47.62097541515849],[  -94.24072265625,  48.28319289548349],[  -95.4052734375,  48.28319289548349],[  -95.4052734375,  47.62097541515849]]],
                [[[  -94.19677734375,  41.27780646738183],[  -93.09814453125,  41.27780646738183],[  -93.09814453125,  42.13082130188811],[  -94.19677734375,  42.13082130188811],[  -94.19677734375,  41.27780646738183]]],
                [[[  -93.71337890625,  37.75334401310656],[  -92.6806640625,  37.75334401310656],[  -92.6806640625,  38.51378825951165],[  -93.71337890625,  38.51378825951165],[  -93.71337890625,  37.75334401310656]]],
                [[[  -90.63720703125,  34.615126683462194],[  -89.47265625,  34.615126683462194],[  -89.47265625,  35.69299463209881],[  -90.63720703125,  35.69299463209881],[  -90.63720703125,  34.615126683462194]]],
                [[[  -93.05419921875,  30.44867367928756],[  -91.77978515625,  30.44867367928756],[  -91.77978515625,  31.57853542647338],[  -93.05419921875,  31.57853542647338],[  -93.05419921875,  30.44867367928756]]],
                [[[  -90.02197265625,  44.276671273775186],[  -88.59374999999999,  44.276671273775186],[  -88.59374999999999,  44.98034238084973],[  -90.02197265625,  44.98034238084973],[  -90.02197265625,  44.276671273775186]]],
                [[[  -90.63720703125,  38.41055825094609],[  -89.49462890625,  38.41055825094609],[  -89.49462890625,  39.18117526158749],[  -90.63720703125,  39.18117526158749],[  -90.63720703125,  38.41055825094609]]],
                [[[  -87.56103515625,  35.62158189955968],[  -86.28662109375,  35.62158189955968],[  -86.28662109375,  36.4566360115962],[  -87.56103515625,  36.4566360115962],[  -87.56103515625,  35.62158189955968]]],
                [[[  -90.63720703125,  31.93351676190369],[  -89.49462890625,  31.93351676190369],[  -89.49462890625,  32.731840896865684],[  -90.63720703125,  32.731840896865684],[  -90.63720703125,  31.93351676190369]]],
                [[[  -69.54345703125,  44.68427737181225],[  -68.5107421875,  44.68427737181225],[  -68.5107421875,  45.336701909968134],[  -69.54345703125,  45.336701909968134],[  -69.54345703125,  44.68427737181225]]],
                [[[  -73.212890625,  41.49212083968776],[  -72.35595703125,  41.49212083968776],[  -72.35595703125,  42.032974332441405],[  -73.212890625,  42.032974332441405],[  -73.212890625,  41.49212083968776]]],
                [[[  -77.93701171875,  38.70265930723801],[  -76.97021484375,  38.70265930723801],[  -76.97021484375,  39.26628442213066],[  -77.93701171875,  39.26628442213066],[  -77.93701171875,  38.70265930723801]]],
                [[[  -79.25537109375,  35.44277092585766],[  -78.15673828125,  35.44277092585766],[  -78.15673828125,  36.13787471840729],[  -79.25537109375,  36.13787471840729],[  -79.25537109375,  35.44277092585766]]],
                [[[  -81.4306640625,  33.55970664841198],[  -80.44189453125,  33.55970664841198],[  -80.44189453125,  34.288991865037524],[  -81.4306640625,  34.288991865037524],[  -81.4306640625,  33.55970664841198]]],
                [[[  -84.90234375,  33.394759218577995],[  -83.91357421875,  33.394759218577995],[  -83.91357421875,  34.19817309627726],[  -84.90234375,  34.19817309627726],[  -84.90234375,  33.394759218577995]]],
                [[[  -82.28759765625,  28.246327971048842],[  -81.2548828125,  28.246327971048842],[  -81.2548828125,  29.209713225868185],[  -82.28759765625,  29.209713225868185],[  -82.28759765625,  28.246327971048842]]],
                [[[  -109.88525390624999,  42.65012181368022],[  -108.56689453125,  42.65012181368022],[  -108.56689453125,  43.50075243569041],[  -109.88525390624999,  43.50075243569041],[  -109.88525390624999,  42.65012181368022]]],
                [[[  -117.61962890624999,  39.04478604850143],[  -116.65283203124999,  39.04478604850143],[  -116.65283203124999,  39.740986355883564],[  -117.61962890624999,  39.740986355883564],[  -117.61962890624999,  39.04478604850143]]],
                [[[  -102.67822265625,  31.42866311735861],[  -101.71142578125,  31.42866311735861],[  -101.71142578125,  32.26855544621476],[  -102.67822265625,  32.26855544621476],[  -102.67822265625,  31.42866311735861]]],
                [[[  -119.47631835937499,  36.03133177633187],[  -118.58642578124999,  36.03133177633187],[  -118.58642578124999,  36.55377524336089],[  -119.47631835937499,  36.55377524336089],[  -119.47631835937499,  36.03133177633187]]],
                [[[  -116.224365234375,  33.091541548655215],[  -115.56518554687499,  33.091541548655215],[  -115.56518554687499,  33.568861182555565],[  -116.224365234375,  33.568861182555565],[  -116.224365234375,  33.091541548655215]]]
            ]
          }
        }
      ]
    },
    "eval_polys": {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "MultiPolygon",
            "coordinates":  [
                [[[-122.13208008,   41.25126946],[-121.37402344,   41.25126946],[-121.37402344,   41.75892074],[-122.13208008,   41.75892074],[-122.13208008,   41.25126946]]],
                [[[-121.15979004,   38.45555349],[-120.23144531,   38.45555349],[-120.23144531,   39.14764411],[-121.15979004,   39.14764411],[-121.15979004,   38.45555349]]],
                [[[-111.80493164,   33.76787602],[-110.83813477,   33.76787602],[-110.83813477,   34.53827854],[-111.80493164,   34.53827854],[-111.80493164,   33.76787602]]],
                [[[-106.125     ,   37.89280344],[-104.74072266,   37.89280344],[-104.74072266,   38.93638677],[-106.125     ,   38.93638677],[-106.125     ,   37.89280344]]],
                [[[-119.08886719,   46.44083284],[-117.63867188,   46.44083284],[-117.63867188,   47.44466731],[-119.08886719,   47.44466731],[-119.08886719,   46.44083284]]],
                [[[-99.84082031,  42.01129149],[-98.50048828,  42.01129149],[-98.50048828,  42.86452395],[-99.84082031,  42.86452395],[-99.84082031,  42.01129149]]],
                [[[-96.6328125 ,  33.20415594],[-95.53417969,  33.20415594],[-95.53417969,  33.97949814],[-96.6328125 ,  33.97949814],[-96.6328125 ,  33.20415594]]],
                [[[-93.44677734,  42.02780647],[-92.34814453,  42.02780647],[-92.34814453,  42.8808213 ],[-93.44677734,  42.8808213 ],[-93.44677734,  42.02780647]]],
                [[[-89.27197266,  45.02667127],[-87.84375   ,  45.02667127],[-87.84375   ,  45.73034238],[-89.27197266,  45.73034238],[-89.27197266,  45.02667127]]],
                [[[-68.79345703,  45.43427737],[-67.76074219,  45.43427737],[-67.76074219,  46.08670191],[-68.79345703,  46.08670191],[-68.79345703,  45.43427737]]],
                [[[-80.68066406,  34.30970665],[-79.69189453,  34.30970665],[-79.69189453,  35.03899187],[-80.68066406,  35.03899187],[-80.68066406,  34.30970665]]],
                [[[-116.86962891,   39.79478605],[-115.90283203,   39.79478605],[-115.90283203,   40.49098636],[-116.86962891,   40.49098636],[-116.86962891,   39.79478605]]]
            ]
          }
        }
      ]
    }
}

In [ ]:
url = f'https://us-central1-skydipper-196010.cloudfunctions.net/ee_tfrecords_training'

headers = {'Content-Type': 'application/json'}

output = requests.post(url, data=json.dumps(payload), headers=headers)
output.json()

**Deserialize Polygons**

In [ ]:
trainFeatures = ee.deserializer.fromJSON(output.json()['training_polygons'])
evalFeatures = ee.deserializer.fromJSON(output.json()['evaluation_polygons'])

**Display Polygons**

In [ ]:
polyImage = ee.Image(0).byte().paint(trainFeatures, 1).paint(evalFeatures, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[38., -100.], zoom_start=5)
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

***
## Predict in Earth Engine
### Python package with the cloud function code

It's necessary to create a Python package to hold the cloud function code.  Here we're going to get started with that by creating a folder for the package.

In [16]:
PACKAGE_PATH = 'Google_Cloud_Functions/ee_model_prediction'

!rm -r {PACKAGE_PATH}
!mkdir {PACKAGE_PATH}
!ls -l {PACKAGE_PATH}

**Files**

Fist we copy the `privatekey.json` file with the [service account keys](https://cloud.google.com/iam/docs/creating-managing-service-account-keys) and the `ee_collection_specifics.py` file into de folder.

In [17]:
!cp ee_collection_specifics.py {PACKAGE_PATH}/ee_collection_specifics.py
!cp {env.privatekey_path} {PACKAGE_PATH}/privatekey.json

**`main.py` file**

We'll use the `%%writefile` command to write the contents of the main function code to a file called `main.py`.

In [18]:
%%writefile {PACKAGE_PATH}/main.py

import ee
import json
import numpy as np
import requests
import ee_collection_specifics

account = 'skydipper@skydipper-196010.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(account, 'privatekey.json')
ee.Initialize(credentials)

def output_image(image, outBands, project_id, model_name, version_name, scale):
    # Load the trained model and use it for prediction.
    model = ee.Model.fromAiPlatformPredictor(
        projectName = project_id,
        modelName = model_name,
        version = version_name,
        inputTileSize = [144, 144],
        inputOverlapSize = [8, 8],
        proj = ee.Projection('EPSG:4326').atScale(scale),
        fixInputProj = True,
        outputBands = {'prediction': {
            'type': ee.PixelType.float(),
            'dimensions': 1,
          }                  
        }
    )
    prediction = model.predictImage(image.toArray()).arrayFlatten([outBands])
    
    return prediction

def ee_tile_url(image, collection, bands):
    # Define the URL format used for Earth Engine generated map tiles.
    EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
    
    dic = {}
    for params in ee_collection_specifics.vizz_params(collection):
        result =  all(elem in bands for elem in params.get('bands'))
        if result:
            mapid = image.getMapId(params)
            
            dic['tile_url_'+str(params.get('bands'))] = EE_TILES.format(**mapid)
    
    return dic

def ee_model_prediction(request):
    request = request.get_json()

    # Variables
    inComposite = request.get('composite') 
    inCollection = request.get('in_collection')
    outCollection = request.get('out_collection')
    inBands = request.get('in_bands')
    outBands = request.get('out_bands')
    startDate = request.get('start')
    stopDate = request.get('end')
    scale = request.get('scale')
    project_id = request.get('project_id')
    model_name = request.get('model_name')
    version_name = request.get('version_name')
    geometry = request.get('geometry')

    # Get input imagery on which it was trained the model
    image = ee.deserializer.fromJSON(inComposite)
    # Select bands and convert them into float
    image = image.select(inBands).float()
        
    # Get output imagery 
    prediction = output_image(image, outBands, project_id, model_name, version_name, scale)
    
    # Clip the prediction area with the polygon
    polygon = ee.Geometry.Polygon(geometry.get('features')[0].get('geometry').get('coordinates'))
    prediction = prediction.clip(polygon)

    # Get centroid
    centroid = polygon.centroid().getInfo().get('coordinates')[::-1]
    
    # Output
    output = {}
    output.update({"centroid": centroid})   
    # Input bands tile urls
    output.update(ee_tile_url(image, inCollection, inBands))
    # Output bands tile urls
    output.update(ee_tile_url(prediction, outCollection, outBands))
    
    # Serialize prediction 
    output.update({'prediction': prediction.serialize()})
    
    return json.dumps(output)

Writing Google_Cloud_Functions/ee_model_prediction/main.py


We include the `requirements.txt` file:

In [19]:
%%writefile {PACKAGE_PATH}/requirements.txt
earthengine-api==0.1.202
numpy==1.17.2
requests==2.21.0

Writing Google_Cloud_Functions/ee_model_prediction/requirements.txt


Finally cd to that directory and deploy the cloud Function with the following command:

`gcloud functions deploy ee_model_prediction --runtime python37 --trigger-http --timeout=120`

#### Call

In [24]:
payload = {
    "composite": output_pre_processing.json()['composite'],
    "in_collection": 'Landsat8_SR',
    "out_collection": 'CroplandDataLayers',
    "in_bands": ['B1','B2','B3','B4','B5','B6','B7'],
    "out_bands": ['cropland', 'land', 'water', 'urban'],
    "start": '2016-01-01',
    "end": '2016-12-31',
    "scale": 30, 
    "project_id": "skydipper-196010",
    "model_name": 'deepvel_Landsat8_Cropland',#'segnet_Landsat8_Cropland',
    "version_name": 'v1576262418',#'v1576136158',
    "geometry": {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [
                  -119.38430786132811,
                  46.07370697571126
                ],
                [
                  -118.89129638671875,
                  46.07370697571126
                ],
                [
                  -118.89129638671875,
                  46.37772792457245
                ],
                [
                  -119.38430786132811,
                  46.37772792457245
                ],
                [
                  -119.38430786132811,
                  46.07370697571126
                ]
              ]
            ]
          }
        }
      ]
    }
}

In [25]:
url = f'https://us-central1-skydipper-196010.cloudfunctions.net/ee_model_prediction'

headers = {'Content-Type': 'application/json'}

output = requests.post(url, data=json.dumps(payload), headers=headers)
pprint(output.json())

{'centroid': [46.2258420319812, -119.1378021240236],
 'prediction': '{"type": "CompoundValue", "scope": [["0", {"type": '
               '"Invocation", "arguments": {"crs": "EPSG:4326"}, '
               '"functionName": "Projection"}], ["1", {"type": "Invocation", '
               '"arguments": {"projection": {"type": "ValueRef", "value": '
               '"0"}, "meters": 30}, "functionName": "Projection.atScale"}], '
               '["2", {"type": "Invocation", "arguments": {"crs": {"type": '
               '"ValueRef", "value": "1"}}, "functionName": "Projection"}], '
               '["3", [144, 144]], ["4", [8, 8]], ["5", {"type": "Invocation", '
               '"arguments": {}, "functionName": "PixelType.float"}], ["6", '
               '{"type": "Invocation", "arguments": {"precision": {"type": '
               '"ValueRef", "value": "5"}}, "functionName": "PixelType"}], '
               '["7", {"type": "Dictionary", "value": {"type": {"type": '
               '"ValueRef", "value"

**Display input and output images**

In [27]:
urls = [s for s in list(output.json().keys()) if "tile_url" in s]
names = [s.strip('tile_url_') for s in urls]

map = folium.Map(location=output.json().get('centroid'), zoom_start=12)

for n, url in enumerate(urls):
    folium.TileLayer(
        tiles=output.json().get(url),
        attr='Google Earth Engine',
        overlay=True,
        name=names[n],
      ).add_to(map)

map.add_child(folium.LayerControl())
map